In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import urllib.request
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
colors = ["#1A3B47", "#FFEA80","#F7AC6F","#E88AA2","#79C5B4","#639FCB","#7874B4"]
empresa_palette = sns.color_palette(colors)

In [ ]:
import glob
inscricoes_file = glob.glob("Workshop*.xlsx")

In [ ]:
consolidado = {"file":"Consolidado de emails.xlsx","sheet":"Consolidado"}
inscritos = {"file":inscricoes_file[0],"sheet":"Endereços"}

In [ ]:
df_consolidado = pd.read_excel(consolidado["file"], consolidado["sheet"] )
df_consolidado.shape

In [ ]:
df_inscritos = pd.read_excel(inscritos["file"],inscritos["sheet"])
df_inscritos.shape

In [ ]:
df_merged = pd.merge(df_inscritos, df_consolidado, how="outer", left_on="Email2", right_on="Email")
df_merged.drop(columns=["Email_x","Name","genre_vowel"], inplace=True)
df_merged.shape

In [ ]:
df_merged.head(3)

In [ ]:
df_merged.columns

In [ ]:
df_merged.rename(columns={"Todo material enviado no kit deverá ser devolvido integralmente até o dia 25 de junho.":"Aceite"}, inplace=True)

In [ ]:
df_accepted = df_merged.dropna(subset=["Aceite"])

In [ ]:
df_accepted.loc[:,'Mailing de'].fillna("Outros")
df_accepted.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'figure.autolayout': True})

ax = sns.countplot(x="Estado", data=df_accepted, palette=empresa_palette
, order=df_merged["Estado"].value_counts().index
)
for p in ax.patches:
    ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+(p.get_width()/2), p.get_height()+0.2));

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'figure.autolayout': True})

ax = sns.countplot(x="Cidade", data=df_accepted, palette=empresa_palette,
order=df_merged["Cidade"].value_counts().index
)
for p in ax.patches:
    ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+(p.get_width()/2), p.get_height()+0.2));

plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'figure.autolayout': True})

ax =sns.countplot(x="Mailing de",data=df_accepted, palette=empresa_palette, 
order=df_accepted["Mailing de"].value_counts().index
)
ax.set_xlabel("Convidado por")

for p in ax.patches:
    ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+(p.get_width()/2), p.get_height()+0.2))

plt.xticks(rotation=90);

plt.savefig("charts/base_total.jpg",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'figure.autolayout': True})
ax = sns.countplot(x="Selecione a melhor data para assistir ao workshop", data=df_accepted)
for p in ax.patches:
    ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+(p.get_width()/2), p.get_height()+0.2))

In [ ]:
df_accepted.columns

In [ ]:
grafico = px.parallel_categories(df_accepted, dimensions=['Mailing de', 'Selecione a melhor data para assistir ao workshop'], width=1400)
grafico.show()


In [ ]:
df_accepted.head(3)

In [ ]:

if "Day" in df_accepted.columns:
    df_accepted.drop(columns=["Day"], inplace=True)

df_dates = df_accepted.copy()
df_dates.loc[:,"Hour"]=df_accepted["Start time"].dt.hour
df_dates.loc[:,"Day"]=df_accepted["Start time"].dt.day
df_dates.loc[:,"Month"]=df_accepted["Start time"].dt.month
df_dates.set_index("Month", inplace=True)


In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'figure.autolayout': True})
sns.lineplot(data=df_dates.groupby(["Day"]).count())